In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import seaborn as sns


from sklearn.model_selection import  train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [4]:
iris = sns.load_dataset("iris")


data = torch.tensor(iris[iris.columns[0:4]].values).float()


labels = torch.zeros(len(data), dtype= torch.long)

labels[iris.species == "setosa"] = 0
labels[iris.species == "versicolor"] = 1
labels[iris.species == "viriginica"] = 2

In [21]:
print(labels)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0])


In [35]:
train_data , test_data , train_labels , test_labels = train_test_split(data, labels, train_size=.8)

tensor_train= TensorDataset(train_data, train_labels)
tensor_test = TensorDataset(test_data, test_labels)

tensor_train = DataLoader(tensor_train,shuffle = True, batch_size=5)
tensor_test = DataLoader(tensor_test)



In [60]:
class my_ann(nn.Module):
  def __init__(self, in_dim, out_dim):
    super().__init__()

    self.lay1 = nn.Linear(in_dim, 15)
    self.lay2 = nn.Linear(15, 15)
    self.lay3 = nn.Linear(15, out_dim)


  def forward(self , x):
    x = self.lay1(x)
    x = F.relu(x)
    x = self.lay2(x)
    x = F.relu(x)
    x = self.lay3(x)

    return x





In [61]:
data_shape_in_fetures = None
for d, l in tensor_train:
  data_shape_in_fetures = d.shape[1]

In [62]:
ann = my_ann(data_shape_in_fetures , 3)


lr = .01
epochs = 1000

loss_fun = nn.CrossEntropyLoss()
optim = torch.optim.SGD(ann.parameters(), lr=lr)
losses = []

In [63]:
for i in range(epochs):
  for d, l in tensor_train:
    y_hat = ann.forward(d)

    loss = loss_fun(y_hat, l)
    losses.append(loss)


    optim.zero_grad()
    loss.backward()
    optim.step()

In [64]:
print(f"First loss: {losses[0]}\nFinal loss: {losses[-1]}")

First loss: 0.9933798909187317
Final loss: 0.003162852954119444


In [81]:
final_pred = []
final_loss = []
acc = []

for d, l in tensor_test:
  pred = ann.forward(d)
  final_pred.append(pred)

  loss = loss_fun(pred, l)
  final_loss.append(loss)

  acc.append(torch.argmax(pred , dim=1).eq(l).sum() / len(l))

In [82]:
# sum(acc.(1.0)) / len(l)

acc_tensor= torch.Tensor(acc)

In [83]:
print(acc_tensor)

tensor([1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])


In [87]:
print (sum(acc_tensor.eq(1.0)) / len(test_labels))

tensor(0.9667)
